# Skimage

Python possesses a powerful library for image analysis: [Skimage](http://scikit-image.org/). An image is represented as a Numpy array (2D for black and white and 3D for color images). Sklearn implements algorithms for all common manipulations of images: filtering, edge detection, transformations, texture detection...

We will give two small examples (mainly to show of the `interact` in IPython notebook).

More examples can be found on the main page.

## 1. Finding blobs

A basis form of image analysis is blob detection: finding regions in the image with different color, intensity... compared to the surroundings. This is often done to, for example, count cells in a microscopy image. Here we will use this to count galaxies in an image.

In [ ]:
from matplotlib import pyplot as plt
from skimage import data  # Skimage contains a set of example images
from skimage.feature import blob_log
from skimage.color import rgb2gray
%matplotlib inline

In [ ]:
image = data.hubble_deep_field()[:600,:600]
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image, interpolation='nearest')  # show image, just treat is as a matrix
image_gray = rgb2gray(image)  # set to grayscale for blob detection

We will use the Laplacian of Gaussian (Log) method to find the galaxies. This methods has threshold parameter that has to be tuned: the higher the beter the specificity but the lower the sensitivity.

First we will create a subroutine to perform the blob detection for a given threshold values:

In [ ]:
def detect_blobs(threshold):
    blobs_log = blob_log(image_gray, max_sigma=30, num_sigma=10, threshold=threshold)
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_title('Number of blobs')
    ax.imshow(image)
    for blob in blobs_log:
        y, x, r = blob
        c = plt.Circle((x, y), r, color='lime', linewidth=2, fill=False)
        ax.add_patch(c)
    print("Detected %i galaxies with a threshold of %s" %(len(blobs_log), threshold))

This function can be used to create a simple widget with a slider using the interact modele.

In [ ]:
from IPython.html.widgets import interact

interact(detect_blobs, threshold=(0.01, 0.5, 0.05))

## 2. Segementation using clustering

Another common task in image analysis is segmentation, i.e. dividing the image into regions with similar propperties. This can be done using $k$-means clustering. 

In [ ]:
from skimage.data import coffee
from sklearn.cluster import KMeans, MiniBatchKMeans
import numpy as np
coffee = coffee()

In [ ]:
image_shape = coffee.shape
print(image_shape)
plt.imshow(coffee)
coffee

So our image is represented by a $400\times600\times3$ array. We can squeeze the image to a datamatrix where the observations (i.e. rows) correspond to the 240000 pixels and the three columns are the RGB values. Subsequently we use $k$-means clustering to find regions with similar color.

In [ ]:
kmeans_clustering = KMeans(n_clusters=10)
%time cluster_kmeans = kmeans_clustering.fit_predict(coffee.reshape(-1, 3))
segm_image_km = kmeans_clustering.cluster_centers_[cluster_kmeans].reshape(image_shape)/255
plt.imshow(segm_image_km)

Nice! But segmentation takes quite long, as $k$-means is applied on a large dataset (240000 pixels!). Sklearn is equipped to deal with large datasets and often there exist algorithms which can be run on multiple processors. For $k$-means clustering there is an example which works by dividing the data into minibatches.

In [ ]:
kmeans_clustering_minbatch = MiniBatchKMeans(n_clusters=10, batch_size=5000)
%time cluster_kmeans_minibatch = kmeans_clustering_minbatch.fit_predict(coffee.reshape(-1, 3))
segm_image_kmmb = kmeans_clustering_minbatch.cluster_centers_[cluster_kmeans].reshape(image_shape)/255
plt.imshow(segm_image_kmmb)

We see that the running time has decreased, but our outcome is rather crappy... 

In [ ]:
@interact
def mini_batch_km_segmentation(n_clusters=(3, 50, 1), batch_size=(100, 100000, 1000)):
    kmeans_clustering_minbatch = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size)
    %time cluster_kmeans_minibatch = kmeans_clustering_minbatch.fit_predict(coffee.reshape(-1, 3))
    segm_image_kmmb = kmeans_clustering_minbatch.cluster_centers_[cluster_kmeans].reshape(image_shape)/255
    plt.imshow(segm_image_kmmb)

# Biopython
[Biopython](http://biopython.org/wiki/Main_Page) contains everything you need for analysing bioinformatics data:
- parsing text-based bioinformatics files
- accessing the most important databases (Uniprot, Genbank, Kegg...)
- studying 3D protein structures
- phylogenetics
- ...

Two small examples to get you started.

## 1. Working with sequences



In [ ]:
from Bio import SeqIO
for item in SeqIO.FastaIO.FastaIterator(open('data/sweet_taste_receptor.txt', 'r')):
    print(item)

In [ ]:
sequence = item.seq
sequence

In [ ]:
sequence.alphabet

Let us see what we can do with one sequence:

In [ ]:
sequence.reverse_complement()

In [ ]:
translated = sequence.translate()
translated

Calculate isoelectric point:

In [ ]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [ ]:
protein_analyser = ProteinAnalysis(str(translated))
print(protein_analyser.isoelectric_point())

## 2. Reading multiple sequence alignments

In [ ]:
from Bio import AlignIO
from Bio.Align import AlignInfo
from collections import Counter
import pandas as pd

In [ ]:
alignment = AlignIO.read("data/muscle_cons_refs.txt", "clustal")
alignment.sort()
print(alignment)

We can calculate a quick concensus sequence:

In [ ]:
align_summary = AlignInfo.SummaryInfo(alignment)
print(align_summary.dumb_consensus(threshold=0.3))

In [ ]:
AA_freq_pos = []
n = alignment.get_alignment_length()

for i in range(n):
    if alignment[0,i] is not '-':
        AA_freq_pos.append(Counter(alignment[1:,i]))

AA_table = pd.DataFrame(AA_freq_pos).T
AA_table = AA_table.fillna(0)  # remove NA
AA_table = AA_table[1:]  # remove empty char

In [ ]:
AA_table /= AA_table.sum()
AA_table

In [ ]:
AA_table.T.plot(figsize=(15, 5))